<a href="https://colab.research.google.com/github/phytometrics/plant_phenotyping_python/blob/main/notebooks/globalwheat2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8を用いた小麦穂検出

<img src=https://cdn.jsdelivr.net/gh/phytometrics/plant_phenotyping_python@main/assets/2660fe2fef4dbe05135476e36c2d9f8f33d7755f79af2850c434acd58c705f24.png width=50%>

小麦穂検出データセットと真値ラベル。David et al., 2021<sup>1</sup>より転載。

1. David, E., Serouart, M., Smith, D., Madec, S., Velumani, K., Liu, S., Wang, X., Pinto, F., Shafiee, S., Tahir, I. S. A., Tsujimoto, H., Nasuda, S., Zheng, B., Kirchgessner, N., Aasen, H., Hund, A., Sadhegi-Tehran, P., Nagasawa, K., Ishikawa, G., … Guo, W. (2021). Global Wheat Head Detection 2021: An Improved Dataset for Benchmarking Wheat Head Detection Methods. Plant Phenomics (Washington, D.C.), 2021, 9846158. https://doi.org/10.34133/2021/9846158

2. https://zenodo.org/record/5092309



2020年版
David, E., Madec, S., Sadeghi-Tehran, P., Aasen, H., Zheng, B., Liu, S., Kirchgessner, N., Ishikawa, G., Nagasawa, K., Badhon, M. A., Pozniak, C., de Solan, B., Hund, A., Chapman, S. C., Baret, F., Stavness, I., & Guo, W. (2020). Global Wheat Head Detection (GWHD) Dataset: A Large and Diverse Dataset of High-Resolution RGB-Labelled Images to Develop and Benchmark Wheat Head Detection Methods. Plant Phenomics (Washington, D.C.), 2020, 3521852. https://doi.org/10.34133/2020/3521852